In [4]:
import sys
sys.path.append("../")
sys.path.append("../../")

In [5]:
import numpy as np

# 将唐诗 json 数据按句提取 方便简单模型处理

In [45]:
raw_data = load_corpus("../../data/chinese-poetry-master/json/poet.song.0.json")

In [46]:
import json

In [47]:
with open("../../data/chinese-poetry-master/json/poet.tang.0.json") as f:
    tangshi0= json.load(f)

In [48]:
from zhconv import convert

In [49]:
with open("../../data/chinese-poetry-master/json/poet.tang.0.json") as f:
        tangshi = json.load(f)

In [50]:
tangshi[0]['paragraphs']

['秦川雄帝宅，函谷壯皇居。', '綺殿千尋起，離宮百雉餘。', '連甍遙接漢，飛觀迥凌虛。', '雲日隱層闕，風煙出綺疎。']

In [51]:
def generate_dataset_for_V1():
    tangshi_set_in_sentence = []
    for i in range(58):

        with open("../../data/chinese-poetry-master/json/poet.tang."+ str(i*1000)+".json") as f:
            tangshis = json.load(f)
        for shi in tangshis:
            for sentence in shi['paragraphs']:
                zh_cn_sentence = convert(sentence, 'zh-cn')
                tangshi_set_in_sentence.append(zh_cn_sentence)
    np.save("../../data/chinese-poetry-master/tangshi_v1.npy", tangshi_set_in_sentence)


generate_dataset_for_V1()
            
        

# 唐诗分为五言和七言

In [10]:
tangshi  =np.load("../../data/chinese-poetry-master/tangshi_v1.npy")

In [17]:
tangshi_5yan = []
tangshi_7yan = []

In [18]:
for sentence in tangshi:
    if len(sentence) > 12:
        tangshi_7yan.append(sentence)
    else:
        tangshi_5yan.append(sentence)

In [21]:
np.save("../../data/chinese-poetry-master/tangshi_v1_5yan.npy", tangshi_5yan)
np.save("../../data/chinese-poetry-master/tangshi_v1_7yan.npy", tangshi_7yan)

In [52]:
tangshi  =np.load("../../data/chinese-poetry-master/tangshi_v1_5yan.npy")

        


In [58]:
tangshi5 =[]
for s in tangshi:
    if len(s) == 12:
        tangshi5.append(s)

In [61]:
np.save("../../data/chinese-poetry-master/tangshi_v1_5yan.npy", tangshi5)

# 读取字嵌入向量，添加标点符号嵌入向量并存储

In [11]:
from zhon.hanzi import punctuation
from nlp_utils import *

In [34]:
char_to_index, index_to_char, index_to_vec = load_char_embeddings("../../data/embedding/sogou")
words,embeddings = load_words_and_embeddings("../../data/embedding/sogou/")

In [30]:
len(words)

364987

In [31]:
embeddings.shape

(364990, 300)

In [43]:
embeddings[341107]

array([-6.89600e-03, -4.32840e-02, -2.19190e-02, -8.41910e-02,
       -3.68050e-02,  2.89730e-02,  1.46620e-02,  1.06170e-02,
       -5.18310e-02,  8.04660e-02, -6.08100e-03, -5.82100e-02,
        7.28930e-02, -4.87340e-02,  1.24399e-01, -9.01360e-02,
        5.24040e-02, -8.13050e-02,  6.25190e-02, -4.57710e-02,
       -6.73040e-02,  5.45010e-02,  1.23135e-01, -3.20800e-03,
        6.56700e-02, -8.93740e-02,  1.81020e-02,  5.13600e-03,
        3.33840e-02,  2.16751e-01,  1.23280e-01, -1.22620e-02,
       -1.77990e-01, -6.51970e-02, -9.00500e-02, -1.17332e-01,
       -1.74990e-02, -3.73660e-02, -3.64100e-03,  8.52650e-02,
        7.33500e-03,  2.38900e-02,  3.91790e-02,  1.22926e-01,
       -2.40790e-02, -2.35110e-02, -1.29441e-01, -9.05890e-02,
       -1.48134e-01,  1.67302e-01,  5.07610e-02, -7.86220e-02,
       -5.67260e-02, -1.45396e-01, -2.97900e-02, -2.22800e-03,
        3.98530e-02, -9.67950e-02, -5.52150e-02,  1.76459e-01,
        1.26694e-01,  3.17600e-03, -9.29760e-02,  1.116

In [44]:
embeddings = np.delete(embeddings, 341107, axis=0)

In [45]:
## 
c_to_ix_plus = {}
ix_to_c_plus = {}
ix_to_v_plus = {}

i = 6892

for index, word in enumerate(words):
    if word in ",.!?，。？！" and word not in char_to_index.keys():
        print(word,i,index)
        c_to_ix_plus[word] = i
        ix_to_c_plus[str(i)] = word
        ix_to_v_plus[str(i)] = embeddings[index]
        i += 1
        

， 6892 0
。 6893 2
？ 6894 54
！ 6895 99
, 6896 1026
. 6897 1878
? 6898 32113
! 6899 49161


In [46]:
char_to_index.update(c_to_ix_plus)
index_to_char.update(ix_to_c_plus)
index_to_vec.update(ix_to_v_plus)

In [48]:
len(index_to_char)

6900

In [47]:
len(index_to_vec)

6900

In [49]:
np.save("../../data/embedding/sogou/char_embedding_plus_punctuation/char_to_index.npy", char_to_index)
np.save("../../data/embedding/sogou/char_embedding_plus_punctuation/index_to_char.npy", index_to_char)
np.save("../../data/embedding/sogou/char_embedding_plus_punctuation/index_to_vec.npy", index_to_vec)

In [ ]:
# one_hot * embedding_matrix  VS  embedding_matrix[index]

len(index_to_vec)

ix = [1,1,1,1,1]
ix[0] = char_to_index['熊']
ix[1] = char_to_index['田']
ix[2] = char_to_index['刘']
ix[3] = char_to_index['易']
ix[4] = char_to_index['林']
one_hot=[]
for index,i in enumerate(ix):
    one_hot.append([0]*6902)
    one_hot[index][i] = 1 

one_hot= np.array(one_hot).T    






embeddings = index_to_vec['0'].reshape(300,1)
temp =index_to_vec['0']
del index_to_vec['0']
len(index_to_vec)

count = 0
embeddings = temp.reshape(300,1)
for i in index_to_vec.values():
    embeddings = np.append(embeddings, np.array(i).reshape(300,1),axis=1)

%%time
for i in range(1000):   
    j = [0] * 5
    j[0] = embeddings[:,ix[0]]
    j[1] = embeddings[:,ix[1]]
    j[2] = embeddings[:,ix[2]]
    j[3] = embeddings[:,ix[3]]
    j[4] = embeddings[:,ix[4]]
    
    

%%time
for i in range(1000):   
    j = []
    i = np.dot(embeddings, one_hot)